In [20]:
from datetime import datetime
import pandas as pd
import requests
import json
import zipfile
import cdflib
import io
import tempfile

In [21]:
DATA_DIR = "data"
TEC_COLLECTION = "SW_OPER_TECATMS_2F"

IBI_COLLECTION = "SW_OPER_IBIATMS_2F"
TEC_MEASUREMENTS = [
    ("gps_position", 3),
    ("leo_position", 3),
    ("prn", 1),
    ("l1", 1),
    ("l2", 1),
]
CDF_EPOCH_1970 = 62167219200000.0

ZIP_TIME_FMT = "%Y%m%dT%H%M%S"
PRE_FETCH = 5000

URL = "https://swarm-diss.eo.esa.int/?do=list&maxfiles={maxfiles}&pos={pos}&file=swarm%2FLevel2daily%2FEntire_mission_data%2F{measurement}%2FTMS%2F{sat}"

URI = "https://swarm-diss.eo.esa.int/?do=download&file="

In [22]:
DATETIME_PSTR = '%Y_%m_%d'
def _decode_time_str(time):
    return datetime.strptime(time, DATETIME_PSTR)

def parse_zip_datetime(zip_file):
    zip_list = zip_file.split('_')

    start_time = datetime.strptime(zip_list[-3], ZIP_TIME_FMT)
    end_time = datetime.strptime(zip_list[-2], ZIP_TIME_FMT)

    return start_time, end_time

In [23]:
start_time = _decode_time_str("2017_01_01")
end_time = _decode_time_str("2017_01_02")
window_size = 120
step_size = 120
prefetch = 5000
bubble_measurements = 'bubble_index'

# Inside the dataset class
maxfiles = 2000
pos = 500
satelite = "Sat_A"

In [24]:
class ViresMetaData(object):
    def __init__(
        self,
        zip_file,
        measurement,
        start_time,
        end_time,
    ):
        self.zip_file = zip_file
        self.measurement = measurement
        self.start_time = start_time
        self.end_time = end_time
        
def get_vires_zip(measurement):
    measurement_url = URL.format(maxfiles=maxfiles, pos=pos, measurement=measurement, sat=satelite)
    print(measurement_url)
    
    resp = requests.get(measurement_url, verify=False)
    
    swarm_datasets = json.loads(resp.content)['results']

    vmds = []
    for swarm_set in swarm_datasets:
        dataset_start_time, dataset_end_time = parse_zip_datetime(swarm_set["name"])
        vmd = ViresMetaData(
            zip_file=swarm_set["path"].replace("\/", "/"),
            measurement=measurement,
            start_time=dataset_start_time,
            end_time=dataset_end_time,
        )
        vmds.append(vmd)
        
    return vmds

In [25]:
tec_vires_zips = get_vires_zip("TEC")

https://swarm-diss.eo.esa.int/?do=list&maxfiles=2000&pos=500&file=swarm%2FLevel2daily%2FEntire_mission_data%2FTEC%2FTMS%2FSat_A


/Users/quinnangill/Desktop/U_Pitt/Summer_Pitt/research/venv/lib/python3.7/site-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'swarm-diss.eo.esa.int'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [26]:
ibi_vires_zips = get_vires_zip("IBI")

https://swarm-diss.eo.esa.int/?do=list&maxfiles=2000&pos=500&file=swarm%2FLevel2daily%2FEntire_mission_data%2FIBI%2FTMS%2FSat_A


/Users/quinnangill/Desktop/U_Pitt/Summer_Pitt/research/venv/lib/python3.7/site-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'swarm-diss.eo.esa.int'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [27]:
def filter_func(variable: ViresMetaData):
    if variable.start_time >= start_time and variable.end_time <= end_time:
        return True
    else:
        return False
    
tec_zip_in_range = list(filter(filter_func, tec_vires_zips))
ibi_zip_in_range = list(filter(filter_func, ibi_vires_zips))

# Extract the Data from Zip files

In [28]:
def get_cdf_tec_data(tec_df, cdf_obj):
    for measurement, values in TEC_MEASUREMENTS:
        if values > 1:
            for i in range(values):
                cdf_meas = cdf_obj[measurement][:, i]
                tec_df[measurement + str(i+1)] = cdf_meas
        else:
            cdf_meas = cdf_obj[measurement]
            tec_df[measurement] = cdf_meas
            
    return tec_df

zips_in_range = [tec_zip_in_range[0], ibi_zip_in_range[0]]

def extract_zip_files(zip_range):
    tec_df = pd.DataFrame()
    ibi_df = pd.DataFrame()
    
    for zip_file in zip_range:
        resp = requests.get(URI + zip_file.zip_file, verify=False)
        
        if not resp.ok:
            raise ValueError(f"ERROR: Unable to get request {URI + zip_file.zip_file}")
        
        cdf_file = zip_file.zip_file.split('/')[-1].replace('ZIP', 'cdf')
        zf = zipfile.ZipFile(io.BytesIO(resp.content), 'r')
        
        try:
            tmp_dir = tempfile.mkdtemp()
            tmp_file = zf.extract(cdf_file, path=tmp_dir)
            cdf_obj = cdflib.CDF(tmp_file)
        except:
            print("----------------------------")
            print("    ERROR: Downloading:")
            print("{}".format(cdf_file))
            print("----------------------------")
            raise ValueError()
            
        if zip_file.measurement == "TEC":
            tec_df["timestamp"] = pd.to_datetime((cdf_obj["Timestamp"] - CDF_EPOCH_1970)/1e3, unit='s')
            tec_df = get_cdf_tec_data(tec_df, cdf_obj)
        else:
            ibi_df["timestamp"] = pd.to_datetime((cdf_obj["Timestamp"] - CDF_EPOCH_1970)/1e3, unit='s')
            ibi_df[bubble_measurements] = cdf_obj[bubble_measurements]
            
    return tec_df, ibi_df
    
tec_df, ibi_df = extract_zip_files(zips_in_range)
# zips_in_range[0].measurement

/Users/quinnangill/Desktop/U_Pitt/Summer_Pitt/research/venv/lib/python3.7/site-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'swarm-diss.eo.esa.int'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/Users/quinnangill/Desktop/U_Pitt/Summer_Pitt/research/venv/lib/python3.7/site-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'swarm-diss.eo.esa.int'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [29]:
small_df = tec_df.head(10000)

In [30]:
def count_groups(grouped_df):
    return len(grouped_df["leo_position1"].unique())

small_df.groupby(by="timestamp").apply(count_groups)
# throw out all that match the time stamps

# if one is different then correct that single data point (if possible)
# otherwise throw away all data at that time step

# if do this then training data can't be dependent on consecutive time step

# a loss of a second is not that big of a deal but with a 1 (plasma bubble that data is important)


timestamp
2017-01-01 00:00:02    1
2017-01-01 00:00:03    1
2017-01-01 00:00:04    1
2017-01-01 00:00:05    1
2017-01-01 00:00:06    1
                      ..
2017-01-01 00:34:29    1
2017-01-01 00:34:30    1
2017-01-01 00:34:31    1
2017-01-01 00:34:32    1
2017-01-01 00:34:33    1
Length: 2071, dtype: int64

In [31]:
small_df.groupby(by="timestamp").apply(count_groups).max()

2

In [39]:
bad_data = small_df.groupby(by="timestamp").apply(count_groups)

In [65]:
idx_good = bad_data.where(lambda x: x > 1).dropna()

In [71]:
idx_good.index[0]

Timestamp('2017-01-01 00:00:37')

In [95]:
cool = [datetime.strftime(gg, '%Y-%m-%d %H:%M:%S') for gg in idx_good.index]

In [99]:
gg_df = small_df[~small_df['timestamp'].isin(cool)]

In [100]:
gg_df

,timestamp,gps_position1,gps_position2,gps_position3,leo_position1,leo_position2,leo_position3,prn,l1,l2
0,2017-01-01 00:00:02,-8.537487e+06,1.456851e+07,2.090384e+07,-1802789.827,1658127.149,6356004.962,2,5.234046e+05,5.234057e+05
1,2017-01-01 00:00:02,-6.472292e+05,2.252916e+07,1.380694e+07,-1802789.827,1658127.149,6356004.962,5,-2.399374e+06,-2.399375e+06
2,2017-01-01 00:00:02,-2.086058e+07,7.817567e+06,1.447835e+07,-1802789.827,1658127.149,6356004.962,6,-2.496468e+06,-2.496472e+06
3,2017-01-01 00:00:02,-1.631289e+07,-2.569429e+06,2.077598e+07,-1802789.827,1658127.149,6356004.962,9,5.222347e+07,5.222346e+07
4,2017-01-01 00:00:02,-7.591596e+06,-1.500507e+07,2.067721e+07,-1802789.827,1658127.149,6356004.962,23,-5.721617e+05,-5.721607e+05
...,...,...,...,...,...,...,...,...,...,...
9995,2017-01-01 00:34:32,1.723452e+07,6.610513e+05,-2.021128e+07,3433638.202,-5305708.478,-2577887.429,10,5.213812e+07,5.213811e+07
9996,2017-01-01 00:34:32,1.799462e+07,-1.716143e+07,-8.850649e+06,3433638.202,-5305708.478,-2577887.429,14,5.231182e+07,5.231182e+07
9997,2017-01-01 00:34:32,-8.552327e+06,-2.355768e+07,-8.419910e+06,3433638.202,-5305708.478,-2577887.429,22,5.064804e+07,5.064803e+07
9998,2017-01-01 00:34:32,1.222844e+07,-2.345963e+07,1.095924e+05,3433638.202,-5305708.478,-2577887.429,27,4.823010e+07,4.823009e+07


In [53]:
small_df.filter(items=list(idx_good),axis=0)

,timestamp,gps_position1,gps_position2,gps_position3,leo_position1,leo_position2,leo_position3,prn,l1,l2


In [32]:
small_df[small_df.timestamp == '2017-01-01 00:00:37']

,timestamp,gps_position1,gps_position2,gps_position3,leo_position1,leo_position2,leo_position3,prn,l1,l2
204,2017-01-01 00:00:37,-8.626618e+06,1.455914e+07,2.087178e+07,-1643371.199,1473466.587,6443894.337,2,6.321532e+05,6.321543e+05
205,2017-01-01 00:00:37,-6.817576e+05,2.247628e+07,1.389137e+07,-1643371.199,1473466.587,6443894.337,5,-2.285078e+06,-2.285079e+06
206,2017-01-01 00:00:37,-2.092352e+07,7.803918e+06,1.439458e+07,-1643371.199,1473466.587,6443894.337,6,-2.315038e+06,-2.315042e+06
207,2017-01-01 00:00:37,-1.626122e+07,-2.646457e+06,2.080673e+07,-1643371.199,1473466.587,6443894.337,9,5.223772e+07,5.223771e+07
208,2017-01-01 00:00:37,-7.535165e+06,-1.507621e+07,2.064424e+07,-1643371.199,1473466.587,6443894.337,23,-7.053759e+05,-7.053750e+05
209,2017-01-01 00:00:37,-7.533556e+06,-1.507824e+07,2.064329e+07,-1638659.244,1467989.867,6446337.987,23,-7.130649e+05,-7.130640e+05
210,2017-01-01 00:00:37,1.390800e+07,6.107132e+06,2.178879e+07,-1643371.199,1473466.587,6443894.337,29,-1.611434e+06,-1.611435e+06


In [33]:
MAX_PRN = 32

def get_columns():
    def expand_measurement(measurement):
        if measurement == "prn":
            return []
        elif "leo" in measurement:
            return [measurement]
        return [measurement + f"_{i+1}" for i in range(MAX_PRN)]
    
    columns = []
    for measurement, values in TEC_MEASUREMENTS:
        if values > 1:
            for i in range(values):
                columns += expand_measurement(measurement + str(i+1))
        else:
            columns += expand_measurement(measurement)
            
    return columns
    
expanded_tec_columns = [
    "gps_position1", "gps_position2", "gps_position3",
    "l1", "l2"
]
expanded_columns = get_columns()

prns_seen = set()

def groupby_func(grouped_df):
    expanded_df = pd.DataFrame(
        columns=expanded_columns
    )
    exp_dict = {}
    for i, row in enumerate(grouped_df.iterrows()):
        row_idx = row[0]
        row_obj = row[1]
        
        
        for j in range(3):
            exp_dict[f"leo_position{j+1}"] = row_obj[f"leo_position{j+1}"]
        prn = row_obj["prn"]
        prns_seen.add(prn)
        for col in expanded_tec_columns:
            exp_dict[f"{col}_{prn}"] = row_obj[col]
    
    expanded_df = expanded_df.append(exp_dict, ignore_index=True)
    
    return expanded_df

example_df = small_df.groupby(by="timestamp").apply(groupby_func)

example_df.head()

,,gps_position1_1,gps_position1_2,gps_position1_3,gps_position1_4,gps_position1_5,gps_position1_6,gps_position1_7,gps_position1_8,gps_position1_9,gps_position1_10,...,l2_23,l2_24,l2_25,l2_26,l2_27,l2_28,l2_29,l2_30,l2_31,l2_32
timestamp,,,,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:02,0,NaN,-8.537487e+06,NaN,NaN,-647229.165683,-2.086058e+07,NaN,NaN,-1.631289e+07,NaN,...,-572160.691230,NaN,NaN,NaN,NaN,NaN,-1.462326e+06,NaN,NaN,NaN
2017-01-01 00:00:03,0,NaN,-8.540035e+06,NaN,NaN,-648212.117560,-2.086238e+07,NaN,NaN,-1.631142e+07,NaN,...,-576145.482958,NaN,NaN,NaN,NaN,NaN,-1.466853e+06,NaN,NaN,NaN
2017-01-01 00:00:04,0,NaN,-8.542583e+06,NaN,NaN,-649195.279516,-2.086418e+07,NaN,NaN,-1.630994e+07,NaN,...,-580125.927988,NaN,NaN,NaN,NaN,NaN,-1.471371e+06,NaN,NaN,NaN
2017-01-01 00:00:05,0,NaN,-8.545131e+06,NaN,NaN,-650178.651563,-2.086599e+07,NaN,NaN,-1.630846e+07,NaN,...,-584102.262717,NaN,NaN,NaN,NaN,NaN,-1.475881e+06,NaN,NaN,NaN
2017-01-01 00:00:06,0,NaN,-8.547679e+06,NaN,NaN,-651162.233711,-2.086779e+07,NaN,NaN,-1.630698e+07,NaN,...,-588074.648322,NaN,NaN,NaN,NaN,NaN,-1.480382e+06,NaN,NaN,NaN


* Searching through all the ones and keep them
* Then where there are ones and max # of prns are (ex) 6 as max (reduce)
* Zero them out (no information about that channel) (zero)

In [34]:
prns_seen

{2, 5, 6, 9, 10, 14, 16, 22, 23, 26, 27, 29, 31}

In [35]:
len(expanded_columns)

163

In [36]:
ibi_df.head(100)

,timestamp,bubble_index
0,2017-01-01 00:00:00,-1
1,2017-01-01 00:00:01,-1
2,2017-01-01 00:00:02,-1
3,2017-01-01 00:00:03,-1
4,2017-01-01 00:00:04,-1
...,...,...
95,2017-01-01 00:01:35,-1
96,2017-01-01 00:01:36,-1
97,2017-01-01 00:01:37,-1
98,2017-01-01 00:01:38,-1



2
5
6
9
23
29


In [37]:
small_df[small_df.timestamp == '2017-01-01 00:00:02']

,timestamp,gps_position1,gps_position2,gps_position3,leo_position1,leo_position2,leo_position3,prn,l1,l2
0,2017-01-01 00:00:02,-8.537487e+06,1.456851e+07,2.090384e+07,-1802789.827,1658127.149,6356004.962,2,5.234046e+05,5.234057e+05
1,2017-01-01 00:00:02,-6.472292e+05,2.252916e+07,1.380694e+07,-1802789.827,1658127.149,6356004.962,5,-2.399374e+06,-2.399375e+06
2,2017-01-01 00:00:02,-2.086058e+07,7.817567e+06,1.447835e+07,-1802789.827,1658127.149,6356004.962,6,-2.496468e+06,-2.496472e+06
3,2017-01-01 00:00:02,-1.631289e+07,-2.569429e+06,2.077598e+07,-1802789.827,1658127.149,6356004.962,9,5.222347e+07,5.222346e+07
4,2017-01-01 00:00:02,-7.591596e+06,-1.500507e+07,2.067721e+07,-1802789.827,1658127.149,6356004.962,23,-5.721617e+05,-5.721607e+05
5,2017-01-01 00:00:02,1.396840e+07,6.031758e+06,2.177105e+07,-1802789.827,1658127.149,6356004.962,29,-1.462325e+06,-1.462326e+06


In [38]:
def groupby_func(var1):
    return len(var1["leo_position3"].unique())

small_df.groupby(by="leo_position1").apply(groupby_func).max()

1

## Normalization

* normalize to expected value of the orbits
* norm of gps_pos1, 2, and 3 ~= 20 million
* leo position norm ~= 550 thousand
* keeps leo sim to gps
* does not get a 0-1 relationship but closer values
* OR treat it and re-weight them to prioritize those anyways


## Nick's Ideas

* targets: iclr and cbr conference workshops
* AI for space
* combine some of what we are doing